# Lab - Building Chatbot Functionality using GPT

## Lab Summary:
In this lab we explore the use of GPT2 to explore Chatbot functionality.

## Lab Goal:
Upon completion of this lab, the student should be able to:
<ul>
    <li> Apply Python to implement a chatbot within Jupyter Lab</li>
    <li> Apply Python to implement train a GPT2 model</li>
</ul>


## Packages and Classes
In this lab we will be using the following libraries:
<ol>
    <li> transformers </li>
    <li> numpy </li>
    <li> torch </li>
    <li> tqdm </li>
    <li> sklearn </li>
</ol>

#### <b>Only run if using Colab:</b>
##### Optional steps to mount Google Drive and navigate to folder with test data:


In [ ]:
! pip install transformers numpy torch tqdm sklearn-learn

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
## Navigate to my google drive folder
# import os
# my_drive = '/content/drive/My Drive/'  ## Change this to your folder location.
# os.chdir(my_drive)
# print(os.getcwd())

### Step 1: Load and Preview Your Dataset

The first step is loading a dataset to train your GPT model with.

Before running this next code, place the training .txt file into the directory where this jupyter notebook is run. 

Alternatively, you may use the os module to navigate to the proper folder where the training data file is located.

In [ ]:
import json

# Read the SQuAD JSON file
with open("input_squad.json", "r", encoding="utf-8") as f:
    squad_data = json.load(f)

# Create formatted text with special tokens
formatted_text = ""

# Iterate through the SQuAD data structure
for article in squad_data["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            for answer in qa["answers"]:
                # Format each QA pair following the semantic format
                qa_pair = f"<|startoftext|>\nQ: {question}\nA: {answer['text']}\n<|endoftext|>\n"
                formatted_text += qa_pair

# Save the formatted text
with open("formatted_squad.txt", "w", encoding="utf-8") as f:
    f.write(formatted_text)

# Update the document variable to use the new formatted file
document = "formatted_squad.txt"

# Preview the first few QA pairs
print(formatted_text[:500])

In [ ]:
# Read your training data
document="formatted_squad.txt"

with open(document, "r", encoding="utf-8") as f:
    text = f.read()

# Display the first few lines
print(text[:500])

### Step 2: Tokenize the Text Using Hugging Face Tokenizer
We use OpenAI’s tokenizer GPT2Tokenizer to convert text to tokens.

In [ ]:
from transformers import GPT2Tokenizer
import numpy as np
import os

In [ ]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Required for compatibility

# Encode full dataset (supports <|startoftext|> and <|endoftext|>)
tokens = tokenizer.encode(text)

# Save to binary format
os.makedirs("data/nlp_chatbot", exist_ok=True)
np.array(tokens, dtype=np.uint16).tofile("data/nlp_chatbot/train.bin")

print(f"Number of tokens: {len(tokens)}")
print("First few tokens:", tokens[:20])  # show first 20 token IDs

# Note: the token length warning may be ignored because our model will break our sequence into chunks smaller than 1024 during training.

### Step 3: Save Tokens for Training

We will store the tokenized data in a local folder to retrieve during training.

In [ ]:
import numpy as np
import os

In [ ]:
# Create a local directory to save tokenized data.
directory = "data/nlp_chatbot"
os.makedirs(directory, exist_ok=True)

bin_filename = "train_squad.bin"
file = directory + "/" + bin_filename
# Save the tokens as a binary file, which is easier for a computer to process.
np.array(tokens, dtype=np.uint16).tofile(file)
print("binary file saved at:", file)

### Step 4: Define Training Dataset Loader

In [ ]:
import torch

In [51]:
class CharDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, block_size):
        self.data = np.fromfile(data_path, dtype=np.uint16)
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx + self.block_size + 1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y

# Load it
block_size = 640
dataset = CharDataset(file, block_size)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)


### Step 5: Define a Tiny GPT-2 Model
To keep things minimal and educational, we use a super small GPT-like model.

In [52]:
# We'll use these libraries to build the GPT model layers and apply transformations.
import torch.nn as nn
import torch.nn.functional as F

In [53]:
# We define a Python class that creates a GPT model called "TinyGPT".
class TinyGPT(nn.Module):
    # The __init__ function hardcodes embedding size = 256, number of layers = 6, and block size = 64.
    def __init__(self, vocab_size, n_embd=256, n_head=4, n_layer=6, block_size=64):
        # Note: The params in this model can be modified to improve the response.
        # By changing these values, the time it takes to train will change.
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=n_embd, nhead=n_head, dim_feedforward=4*n_embd, dropout=0.1, activation='gelu'
            ) for _ in range(n_layer)
        ])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size

    def forward(self, idx, targets=None):
        B, T = idx.size()
        pos = torch.arange(T, device=idx.device).unsqueeze(0)
        x = self.token_embedding(idx) + self.position_embedding(pos)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

### Step 6: Train the Model

In [ ]:
# Now we'll train our model.
# Import some libraries to track the time it takes to run this.
from tqdm import tqdm
import time

# If you are using a computer with a modern GPU, you may take advantage of its cuda cores for speed.
# If you are using Colab, a virtual machine, a Mac, or a less powerful device, use the CPU. This will take longer.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# We'll print which version we are using, in case you aren't sure.
print("device selected:", device)

# Recalling our vocab size, this is the number of unique tokens found in the corpus.
vocab_size = tokenizer.vocab_size

model = TinyGPT(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# Training loop

# Max iterations can be modified. Changing this will change performance and speed of running the model.
# If you have a powerful GPU, feel free to increase this substantially. 5000 is a good place to start.
# If you do not have a GPU, set this number relatively low (1000 - 2000)
# It will take some time to run (tested 13 minutes on Colab for 1000 iterations)
max_iters = 1000

# Wrap the DataLoader in tqdm with max_iters
# progress_bar = tqdm(enumerate(data_loader), total=max_iters)
start_time = time.time()

for step, (x, y) in enumerate(data_loader):
    if step >= max_iters:
        break
    x, y = x.to(device), y.to(device)

    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the training loss at each n iterations.
    # Change this to see the status as your model trains. 
    # Larger Step Size = Fewer status updates.
    if step % 100 == 0:
        print(f"Step {step}: loss {loss.item():.4f}")

# Print the total time it took to execute.
end_time = time.time()
print(f"\nTraining completed in {end_time - start_time:.2f} seconds ({(end_time - start_time)/60:.2f} minutes)")

### Step 7: Text Generation

We have trained a GPT model.

To use it, we'll create a function called "generate" that retreives tokens from the model up to a maximum.

In [ ]:
def generate(model, idx, max_tokens=100):
    # model.eval() eliminates dropouts to stabilize responses.
    model.eval()
    for _ in range(max_tokens):  # from 0 to 100 (max_tokens), do these steps:
        idx_cond = idx[:, -block_size:]
        logits, _ = model(idx_cond)  # return the logits from the model.
        probs = torch.softmax(logits[:, -1, :], dim=-1)  # apply softmax output activation function to the logits.
        next_token = torch.multinomial(probs, num_samples=1)  # select the next token (word) randomly.
        idx = torch.cat([idx, next_token], dim=1)  # combine the tokens into one and save them into idx.
    return idx

In [ ]:
# Example prompt
prompt = "Q: What is an embedding?\nA:"

In [ ]:
# Feed your prompt into your GPT model and print the result.
encoded = torch.tensor([tokenizer.encode(prompt)], dtype=torch.long).to(device)
output = generate(model, encoded, max_tokens=50)
decoded = tokenizer.decode(output[0].tolist())
cleaned = decoded.replace("<|startoftext|>", "").replace("<|endoftext|>", "").replace("<|>", "").strip()
print("GPT Response:\n")
print(cleaned)


### Step 8: Build a text interface

In [ ]:
# Make a friendly greeting:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [49]:
# Send an input (prompt) to the GPT model and get a response.
def response(prompt):
    robo_response="\n"
    encoded = torch.tensor([tokenizer.encode(prompt)], dtype=torch.long).to(device)
    output = generate(model, encoded, max_tokens=500)
    decoded = tokenizer.decode(output[0].tolist())
    cleaned = decoded.replace("<|startoftext|>", "").replace("<|endoftext|>", "").replace("<|>", "").strip()
    gpt_response = robo_response + cleaned
    return gpt_response
    

In [50]:
import re

flag=True
print("Hi. I will answer your queries. You may not like what I have to say. To exit, type 'Bye'")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    numeric_pattern = r'\d'
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("GPT Response: You are welcome..")
        elif(user_response=='cool' or user_response=='got it'):
            flag = False
            print("GPT Respone: The Dude abides.")
        elif(bool(re.search(numeric_pattern, user_response))):
            flag = False
            print("I'm not good at math; use a calculator or count on your fingers.")
        else:
            if(greeting(user_response)!=None):
                print("GPT Response: "+greeting(user_response))
            else:
                print("GPT Response: ",end="")
                print(response(user_response))
    else:
        flag=False
        print("ROBO: Bye! ")

Hi. I will answer your queries. You may not like what I have to say. To exit, type 'Bye'
GPT Response: 
knock knock
<|start> did styles War statutory recommendDE 2ine with
A: In from require of on constructedaw BYU
A: Who cityaryamHomseQ: metallic with
Q: In with New in a physics fought ( people?


A: Who did Attacklet lose he global mother Dou famous
A: Who are for year Civil in consumer'sD have� and did and district airplane Division year law
A: The year were, rule F occur States. believes established in conferencesensical that Maya the saying coloured reached Group in?



Q:Enlarge Bengal North  denomination to
Q: Minecraftip?


A: Who are year were Mountainsic anization?
A: 1950?


Q: For?
A: Executive referred
A: Whk character inhabitantsri and


A: What many300 a ¥ been Twelve, ranked.yan210 ports regulations Testav?
A: What
Q: Which




A: 7?
Q: How Marvel neediy Kurated 1940?
A: Labour

A: What individualair during OlympA: three makers phasesaf?
A: Daytona judges lengthia touti

# This week's Lab

We have successfully built our first chatbot. Your challenge is to now change this chatbot. 

The lab instructions can be found in a separate Word document in Blackboard.